<a href="https://colab.research.google.com/github/yuto90/whisper_gradio/blob/main/gijiroku_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# whisper用
!pip install git+https://github.com/nyanta012/whisper -q
# chatGPT用
!pip install openai

In [ ]:
!pip install gradio==3.4.0

In [ ]:
import gradio as gr 
import whisper

model = whisper.load_model("large")

In [ ]:
import openai

def generate_gijiroku(video_name):
    transcribtfile = create_transcribtfile(video_name)
    summaryfile = create_summaryfile(transcribtfile)

    return summaryfile

# 文字起こしファイルを作成する
def create_transcribtfile(video_name):
    results = model.transcribe(video_name, verbose=False, language="ja")

    # アウトプット用ファイルを上書きモードで作成して開く
    with open("transcribe.txt", mode="w") as f:
        # 文字起こし結果を書き込み
        for index, _dict in enumerate(results["segments"]):
            f.write(f'{_dict["text"]}\n')
    return "transcribe.txt"

# 要約ファイルを作成
def create_summaryfile(transcript):
    openai.api_key = ""  # APIキー
    
    system_template = """会議の書き起こしが渡されます。
    
    この会議のサマリーをMarkdown形式で作成してください。サマリーは、以下のような形式で書いてください。
    
    - 会議の目的
    - 会議の内容
    - 会議の結果"""
    
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_template},
            {"role": "user", "content": transcript}
        ]
    )
    response_text = completion.choices[0].message.content
    
    # アウトプット用ファイルを上書きモードで作成して開く
    with open("summary.txt", mode="w") as f:
        # 文字起こし結果を書き込み
        f.write(f'{response_text}')
    return "summary.txt"

css = """
        .gradio-container {
            font-family: 'IBM Plex Sans', sans-serif;
        }
        .container {
            max-width: 730px;
            margin: auto;
            padding-top: 1.5rem;
        }
        #gallery {
            min-height: 22rem;
            margin-bottom: 15px;
            margin-left: auto;
            margin-right: auto;
            border-bottom-right-radius: .5rem !important;
            border-bottom-left-radius: .5rem !important;
        }
        #gallery>div>.h-full {
            min-height: 20rem;
        }

"""

block = gr.Blocks(css=css)

with block:   
    gr.Markdown("議事録自動作成アプリ")
    with gr.Group():
        with gr.Box():
            with gr.Row(mobile_collapse=False, equal_height=True):
                text_button = gr.Button("transcribe").style(
                        margin=False,
                        rounded=(True, True, True, True),
                    )
        video = gr.Audio(source="upload", type="filepath", interactive=True)       
        transcribe = gr.File(label="summary text file")
        # ボタンを押した時の挙動
        text_button.click(generate_gijiroku, inputs=[video], outputs=[transcribe])

block.launch(debug=True)